In [ ]:
# !conda install swig
# !pip install stable-baselines3[extra]
# !pip install gymnasium
# !pip install stable_baselines3
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install gymnasium[box2d]
# !pip install sb3-contrib
# !pip install tensorboard 
#!pip install scikit-learn 


In [ ]:
import os
import gymnasium as gym
from gymnasium.spaces import Discrete, Box
import tensorboard
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import A2C
from sb3_contrib import RecurrentPPO
from TraderEnv import TraderEnv
from DataProvider import DataProvider
import matplotlib.pyplot as plt


In [ ]:
log_path = os.path.join('Training', 'Logs')
PPO_Path = os.path.join('Training', 'SavedModels', 'PPO_Model_Cartpole')

In [ ]:
data_provider = DataProvider('Data/Binance_BTCUSDT_2023_minute.csv')
df_raw = data_provider.get_raw_data()[110_000:400_000]



In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_raw['close'], label='Close Price')
plt.title('Close Price Chart')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
trade_env = TraderEnv(df_raw, trade_size_dollars=9_000, initial_capital=10_000)


In [ ]:
# Wrap the environment
env = DummyVecEnv([lambda: trade_env])

# Initialize the model
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)
# model = PPO.load(PPO_Path+"5002", env=env)

# Train the model

In [ ]:
model.learn(total_timesteps=10_000_000)

In [ ]:
model.save(PPO_Path+"5003")


In [ ]:
# trade_env.reset()

In [ ]:
# env.reset()

In [ ]:
# env.render()

In [ ]:
test_env = TraderEnv(df_raw[:50000], trade_size_dollars=9_000, initial_capital=10_000)
test_env = DummyVecEnv([lambda: test_env])

episodes = 5
for episode in range(1, episodes + 1):
    obs = test_env.reset()
    done = False
    score = 0
    
    while not done:
        # test_env.render(mode="human")
        action, _ = model.predict(obs)
        obs, reward, done, info = test_env.step(action)
        score += reward

    env_info = info[0]
    current_capital = env_info.get('current_capital', 'N/A')
    sharpe_ratio = env_info.get('sharpe_ratio', 'N/A')
    current_step = env_info.get('current_step', 'N/A')
    trades_amount = env_info.get('trades_amount', 'N/A')
    print(f'Episode: {episode} Score: {score} Current Capital: {current_capital} Sharpe Ratio: {sharpe_ratio} Current step: {current_step} Trades amount: {trades_amount}')

test_env.close()


In [ ]:
# env = gym.make("LunarLander-v2", render_mode="human")
# observation, info = env.reset(seed=42)
# for _ in range(1000):
#    action = env.action_space.sample()  # this is where you would insert your policy
#    observation, reward, terminated, truncated, info = env.step(action)
#    env.render()

#    if terminated or truncated:
#       observation, info = env.reset()

# env.close()


In [ ]:
# enviornment_name = "CartPole-v1"

In [ ]:
# env = gym.make(enviornment_name, render_mode="human") # human  rgb_array
# env = DummyVecEnv([lambda: env])

In [ ]:
# # model = RecurrentPPO('MlpLstmPolicy', env, verbose=1, tensorboard_log=log_path)
# model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)


In [ ]:
# model.learn(total_timesteps=30_000)

In [ ]:
# del model

In [ ]:
# PPO_Path

In [ ]:
# evaluate_policy(model, env, n_eval_episodes=1, render=True)

In [ ]:
# episodes = 5
# for episode in range(1, episodes+1):
#     obs = env.reset()
#     done = False
#     score = 0
    
#     while not done:
#         env.render()
#         action, _ = model.predict(obs)
#         obs, reward, done, info = env.step(action)
#         score += float(reward)
#     print('Episode:{} Score:{}'.format(episode, score))
# env.close()

In [ ]:
# training_log_path = os.path.join(log_path, 'PPO_6')

In [ ]:
# !tensorboard --logdir={training_log_path}